In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from scipy import interp
import os
from data_accuracy import *


%matplotlib inline

In [17]:
from scipy import integrate

# Loding the original annotated text file

In [6]:
df_r=pd.read_csv('/home/ce/Documents/py-faster-rcnn/Accuracy_Check/annotationTools/sampleResults/results29-90.csv',sep=';',header=None,dtype=str)

In [7]:
df_tr=pd.read_csv('/home/ce/Documents/py-faster-rcnn/Accuracy_Check/annotationTools/sampleResults/stop-split2.csv',sep=';',header=None,dtype=str)

In [4]:
test_file = raw_input("Enter the original annotated txt file with path: ")
df_test=pd.read_csv(test_file,sep=';',header=None,dtype=str)
print 'Number of files in which signs are annotated',len(np.unique(df_test[0]))

Enter the original annotated txt file with path: /home/ce/Documents/py-faster-rcnn/data/GTSDB/data/train_gtsdb.txt
Number of files in which signs are annotated 297


# Loading the predicted text file

In [5]:
test_file_pred = raw_input("Enter the predicted txt file with path: ")
df_test_pred=pd.read_csv(test_file_pred,sep=';',header=None) #output of the test files
print 'Number of files in which signs are detected',len(np.unique(df_test_pred[0]))

Enter the predicted txt file with path: /home/ce/Documents/py-faster-rcnn/Accuracy_Check/gt.txt
Number of files in which signs are detected 741


In [3]:
# Updating the BBox which are in float to int
df_test_pred[1]=df_test_pred[1].apply(lambda x:get_bbox(x))
print 'Bbox in float is updated to int'

Bbox in float is updated to int


In [ ]:
print 'Merging duplicates...'
print len(df_test_pred)
df_test_pred.drop(merge_duplicates(df_test_pred),inplace=True)
print len(df_test_pred)

In [160]:
df_pred = pd.concat([df_test_pred.drop([1,3], axis=1), df_test_pred[1].apply(pd.Series)], axis=1)
df_pred.head()

,0,2,0,1,2,3
0,00012,Speed Limit 50,976,365,1041,418
1,00061,Speed Limit 80,1207,334,1288,409
2,00064,Speed Limit 50,784,447,802,465
3,00064,Speed Limit 50,776,455,796,478
4,00064,Speed Limit 50,789,463,806,481


In [29]:
df_pred.columns=[0,1,2,3,4,5]
df_pred=df_pred[df_pred[1]==raw_input('Enter the class: ')]

df_pred[0]=df_pred[1]+'_'+df_pred[0]
df_pred=df_pred.drop(1,axis=1)
df_pred.head()

NameError: name 'df_pred' is not defined

In [ ]:
df_test_anno=df_test
df_test_anno.columns=[0,2,3,4,5,1]
df_test_anno.to_csv('/home/ce/Documents/py-faster-rcnn/Accuracy_Check/annotationTools/input_annotations.csv',sep=';',header=True,index=False)
df_test[10:].to_csv('/home/ce/Documents/py-faster-rcnn/Accuracy_Check/annotationTools/output_pred.csv',sep=';',header=True,index=False)

In [46]:
print 'Accuracy for detection....'

Accuracy for detection....


In [47]:
for d in [df_test_pred]:
    tp=0
    fp=0
    fn=0
    for f in np.unique(d[0]):
        d1=df_test[df_test[0]==f]
        
        d2=d[d[0]==f]      

        flag=np.ones(len(d1))
        for i in d2.iterrows():
            flag2=0
            i=np.array(i)[1]
            bbox1=get_bbox(i[1])
            start=0
            for j in d1.iterrows():
                j=np.array(j)[1]
                bbox2=get_bbox(list(j[1:5]))
                if get_overlap(bbox1,bbox2)>0.5:
                    tp+=1
                    flag[start]=0
                    flag2=1
                start+=1

            if flag2==0:
                fp+=1

        fn+=np.sum(flag)

    print 'tp',tp,'  fp', fp,'   fn',fn
    print 'precison',float(tp)/(tp+fp)
    print 'recall',float(tp)/(tp+fn)

IndexError: list index out of range

In [ ]:
fpr, tpr, thresholds = roc_curve(y[test], probas_[:, 1])
roc_auc = auc(fp, tp)
print 'Area under the curve: ' + str(roc_auc)

In [ ]:
print 'Accuracy for recognition...'

In [ ]:
list1=[]
score=0.5
for l in training_labels:
    tp=0
    fp=0
    fn=0
    counter=0
    counter1=0
    df_final1=df_test_pred[df_test_pred[2]==l]
    df_test1=df_final1[df_final1[3]>score].copy()  #use df_test or df
    for f in np.unique(df_test1[0]):
        d1=df_test[df_test[0]==f]
        #d2=df[df[0]==f]
        d2=df_test1[df_test1[0]==f]
        counter+=len(d1)
        flag=np.ones(len(d1))

        for i in d2.iterrows():
            flag1=0
            counter1+=1
            i=np.array(i)[1]
            bbox1=get_bbox(i[1])
            start=0
            for j in d1.iterrows():
                j=np.array(j)[1]
                bbox2=get_bbox(list(j[1:5]))
                if get_overlap(bbox1,bbox2)>0.5:
                    if (i[2]==j[5]) :
                        tp+=1
                        flag[start]=0
                        flag1=1

            if flag1==0:
                fp+=1

        fn+=int(np.sum(flag))

    if float(tp)/(tp+fn) <0.5:
        print counter, 'should be equal to tp+fn=',tp+fn
        print counter1, 'should be equal to tp+fp=',tp+fp
        print tp,fp,fn
        print l,score,'precison',float(tp)/(tp+fp),'recall',float(tp)/(tp+fn),'\n'
        list1.append(l)

In [21]:
auc=integrate.cumtrapz(range(44),range(44))*100

In [23]:
aucs=[]
aucs.append(auc[-1])